In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

home_team_spread
2.5    687935
Name: count, dtype: int64

In [106]:
df = pd.read_csv('team_box.csv')
df = df.drop(columns=['Unnamed: 0'])
df.loc[(df['team_id'] == 17), 'team_name'] = 'Aces'
df.loc[(df['team_id'] == 17), 'team_short_display_name'] = 'Aces'
df.loc[(df['team_id'] == 17), 'team_abbreviation'] = 'LV'
df.loc[(df['team_id'] == 17), 'team_location'] = 'Las Vegas'
df.loc[(df['opponent_team_id'] == 17), 'opponent_team_name'] = 'Aces'
df.loc[(df['opponent_team_id'] == 17), 'opponent_team_short_display_name'] = 'Aces'
df.loc[(df['opponent_team_id'] == 17), 'opponent_team_abbreviation'] = 'LV'
df.loc[(df['opponent_team_id'] == 17), 'opponent_team_location'] = 'Las Vegas'
df = df.loc[(df['team_id'] < 80) & (df['opponent_team_id'] < 80)]
df = df.sort_values(by=['team_id', 'game_date']).reset_index(drop=True)
df['season_game_n'] = df.groupby(['team_id', 'season']).cumcount() + 1
df = df.loc[(df['season_type'] == 2)]
data = pd.DataFrame()

for i in range(17, 25):
    data_ = pd.read_csv(f'data/box/box_{i}.csv')
    data = pd.concat([data, data_], axis=0)

minutes_per_game = data.groupby(['game_id','team_id'], as_index=False)['minutes'].sum()
minutes_per_game['minutes'] = ((minutes_per_game['minutes'] / 25).round() * 25)


df = pd.merge(df, minutes_per_game, on=['game_id', 'team_id'], how='left')


df['game_date'] = pd.to_datetime(df['game_date'])

# Sort by team_id and game_date
df = df.sort_values(by=['team_id', 'game_date']).reset_index(drop=True)

df['days_since_last_game'] = df.groupby(['team_id', 'season'])['game_date'].diff().dt.days

# Set to None if the previous game occurred in a different season or if it's the first game
df['days_since_last_game'] = df.groupby(['team_id', 'season']).apply(
    lambda group: group.apply(
        lambda row: None if pd.isna(row['days_since_last_game']) else row['days_since_last_game'], 
        axis=1
    )
).reset_index(drop=True)

print(df)

        game_id  season  season_type  game_date       game_date_time  team_id  \
0     400927396    2017            2 2017-05-14  2017-05-14 18:00:00        3   
1     400927405    2017            2 2017-05-20  2017-05-20 20:00:00        3   
2     400927412    2017            2 2017-05-25  2017-05-25 12:00:00        3   
3     400927417    2017            2 2017-05-27  2017-05-27 22:00:00        3   
4     400927422    2017            2 2017-05-30  2017-05-30 20:00:00        3   
...         ...     ...          ...        ...                  ...      ...   
3265  401620429    2024            2 2024-09-10  2024-09-10 19:30:00       20   
3266  401620436    2024            2 2024-09-13  2024-09-13 19:30:00       20   
3267  401620443    2024            2 2024-09-15  2024-09-15 15:00:00       20   
3268  401620449    2024            2 2024-09-17  2024-09-17 19:30:00       20   
3269  401620453    2024            2 2024-09-19  2024-09-19 19:00:00       20   

            team_uid      t

/var/folders/p9/tjxg6pk13wd5gkgl1h535fm80000gn/T/ipykernel_47164/2910374200.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['days_since_last_game'] = df.groupby(['team_id', 'season']).apply(


In [109]:
team_to_airport = {
    "SEA": "SEA",  # Seattle Storm -> Seattle-Tacoma International Airport
    "CHI": "ORD",  # Chicago Sky -> O'Hare International Airport
    "CONN": "BDL", # Connecticut Sun -> Bradley International Airport
    "LA": "LAX",   # Los Angeles Sparks -> Los Angeles International Airport
    "ATL": "ATL",  # Atlanta Dream -> Hartsfield-Jackson Atlanta International Airport
    "PHX": "PHX",  # Phoenix Mercury -> Phoenix Sky Harbor International Airport
    "WSH": "DCA",  # Washington Mystics -> Ronald Reagan Washington National Airport
    "MIN": "MSP",  # Minnesota Lynx -> Minneapolis-Saint Paul International Airport
    "NY": "JFK",   # New York Liberty -> John F. Kennedy International Airport
    "DAL": "DFW",  # Dallas Wings -> Dallas/Fort Worth International Airport
    "LV": "LAS",   # Las Vegas Aces -> Harry Reid International Airport
    "IND": "IND"   # Indiana Fever -> Indianapolis International Airport
}

distance_matrix = {
    "ATL": {"ATL": 0, "ORD": 606, "BDL": 947, "DFW": 731, "IND": 457, "LAS": 1749, "LAX": 1941, "MSP": 907, "JFK": 760, "PHX": 1585, "SEA": 2181, "DCA": 543},
    "ORD": {"ATL": 606, "ORD": 0, "BDL": 777, "DFW": 802, "IND": 182, "LAS": 1520, "LAX": 1745, "MSP": 334, "JFK": 740, "PHX": 1440, "SEA": 1713, "DCA": 612},
    "BDL": {"ATL": 947, "ORD": 777, "BDL": 0, "DFW": 1418, "IND": 761, "LAS": 2300, "LAX": 2486, "MSP": 1050, "JFK": 108, "PHX": 2147, "SEA": 2487, "DCA": 313},
    "DFW": {"ATL": 731, "ORD": 802, "BDL": 1418, "DFW": 0, "IND": 761, "LAS": 1050, "LAX": 1240, "MSP": 852, "JFK": 1388, "PHX": 868, "SEA": 1678, "DCA": 1185},
    "IND": {"ATL": 457, "ORD": 182, "BDL": 761, "DFW": 761, "IND": 0, "LAS": 1630, "LAX": 1820, "MSP": 517, "JFK": 648, "PHX": 1480, "SEA": 1820, "DCA": 515},
    "LAS": {"ATL": 1749, "ORD": 1520, "BDL": 2300, "DFW": 1050, "IND": 1630, "LAS": 0, "LAX": 236, "MSP": 1320, "JFK": 2250, "PHX": 255, "SEA": 857, "DCA": 2100},
    "LAX": {"ATL": 1941, "ORD": 1745, "BDL": 2486, "DFW": 1240, "IND": 1820, "LAS": 236, "LAX": 0, "MSP": 1530, "JFK": 2475, "PHX": 370, "SEA": 960, "DCA": 2300},
    "MSP": {"ATL": 907, "ORD": 334, "BDL": 1050, "DFW": 852, "IND": 517, "LAS": 1320, "LAX": 1530, "MSP": 0, "JFK": 1020, "PHX": 1260, "SEA": 1396, "DCA": 930},
    "JFK": {"ATL": 760, "ORD": 740, "BDL": 108, "DFW": 1388, "IND": 648, "LAS": 2250, "LAX": 2475, "MSP": 1020, "JFK": 0, "PHX": 2150, "SEA": 2420, "DCA": 214},
    "PHX": {"ATL": 1585, "ORD": 1440, "BDL": 2147, "DFW": 868, "IND": 1480, "LAS": 255, "LAX": 370, "MSP": 1260, "JFK": 2150, "PHX": 0, "SEA": 1100, "DCA": 2000},
    "SEA": {"ATL": 2181, "ORD": 1713, "BDL": 2487, "DFW": 1678, "IND": 1820, "LAS": 857, "LAX": 960, "MSP": 1396, "JFK": 2420, "PHX": 1100, "SEA": 0, "DCA": 2320},
    "DCA": {"ATL": 543, "ORD": 612, "BDL": 313, "DFW": 1185, "IND": 515, "LAS": 2100, "LAX": 2300, "MSP": 930, "JFK": 214, "PHX": 2000, "SEA": 2320, "DCA": 0}
}


df["airport"] = df.apply(
    lambda row: team_to_airport[row["team_abbreviation"]] if row["team_home_away"] == "Home" else team_to_airport[row["opponent_team_abbreviation"]],
    axis=1
)

df = df.sort_values(by=["team_abbreviation", "game_date"])

# Add last_airport column by shifting current_airport
# df["last_airport"] = df.groupby("team_abbreviation")["current_airport"].shift(1)

# # Fill NaN for the first game of each team (no last airport)
# df["last_airport"] = df["last_airport"].fillna("N/A")

def get_distance(last_airport, current_airport):
    if last_airport == "N/A":
        return 0  # No distance for the first game
    return distance_matrix[last_airport][current_airport]

# # Add distance column
# df["distance_travelled"] = df.apply(
#     lambda row: get_distance(row["last_airport"], row["current_airport"]),
#     axis=1
# )



df = df.sort_values(by=["team_id", "game_date"])

# Function to calculate rest pattern and score
def calculate_distance_per_day(group):
    group = group.reset_index(drop=True)
    distances = []
    days_between_games = []
    distance_per_day = []
    
    for i in range(len(group)):
        if i == 0:
            distances.append(0)  # First game has no distance
            days_between_games.append(0)
            distance_per_day.append(0)
            continue
        
        # Calculate distance between consecutive games
        prev_airport = group.loc[i-1, "airport"]
        curr_airport = group.loc[i, "airport"]
        distance = distance_matrix[prev_airport][curr_airport]
        distances.append(distance)
        
        # Calculate days between consecutive games
        days = (group.loc[i, "game_date"] - group.loc[i-1, "game_date"]).days
        days_between_games.append(days)
        
        # Calculate distance per day
        if days == 0:
            distance_per_day.append(0)  # Avoid division by zero
        else:
            distance_per_day.append(distance / days)
    
    group["distance_travelled"] = distances
    group["days_between_games"] = days_between_games
    group["distance_per_day"] = distance_per_day
    
    # Calculate rolling average of distance per day over last 5 games
    group["avg_distance_per_day_last_5"] = group["distance_per_day"].rolling(window=5, min_periods=1).mean()
    
    
    return group

df = df.groupby(['season',"team_id"]).apply(calculate_distance_per_day).reset_index(drop=True)




/var/folders/p9/tjxg6pk13wd5gkgl1h535fm80000gn/T/ipykernel_47164/954620297.py:100: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['season',"team_id"]).apply(calculate_distance_per_day).reset_index(drop=True)


In [111]:
df['avg_distance_per_day_last_5'].describe()

count    3270.000000
mean      374.085706
std       176.974051
min         0.000000
25%       259.864643
50%       368.813333
75%       483.488333
max      1096.933333
Name: avg_distance_per_day_last_5, dtype: float64

In [126]:
import pandas as pd

def calculate_possessions(df):
    """
    Calculate possessions for each team using the formula:
    Possessions = FGA - OR + TO + 0.475 * FTA
    """
    df['possessions'] = (
        df['field_goals_attempted'] - df['offensive_rebounds'] + 
        df['turnovers'] + 0.475 * df['free_throws_attempted']
    )
    return df

def calculate_tempo(df):
    """
    Calculate tempo (possessions per 40 minutes).
    """
    df['tempo'] = (df['possessions'] / (df['minutes']/5)) * 40
    return df

def calculate_efficiency(df):
    """
    Calculate offensive and defensive efficiency.
    """
    df['ORtg'] = (df['team_score'] / df['possessions']) * 100
    df['DRtg'] = (df['opponent_team_score'] / df['possessions']) * 100
    df['EM'] = df['ORtg'] - df['DRtg']
    return df

def calculate_adjusted_metrics(df, alpha=0.25):
    """
    Calculate adjusted tempo, offensive efficiency, and defensive efficiency for each game.
    Use a weighting factor (alpha) to flatten the adjustments.
    """
    # Calculate dynamic national averages
    df['national_avg_tempo'] = df['tempo'].expanding().mean()
    df['national_avg_ORtg'] = df['ORtg'].expanding().mean()
    df['national_avg_DRtg'] = df['DRtg'].expanding().mean()

    # Merge opponent stats
    df = df.merge(
        df[['game_id', 'team_id', 'tempo', 'ORtg', 'DRtg']],
        how='left',
        left_on=['game_id', 'opponent_team_id'],
        right_on=['game_id', 'team_id'],
        suffixes=('', '_opponent')
    )

    # Adjust tempo with weighting factor
    df['adjT'] = df['national_avg_tempo'] + alpha * (
        (df['tempo'] - df['national_avg_tempo']) + (df['tempo_opponent'] - df['national_avg_tempo'])
    ) / 2

    # Adjust ORtg and DRtg with weighting factor
    df['adjO'] = df['national_avg_ORtg'] + alpha * (
        (df['ORtg'] - df['national_avg_ORtg']) + (df['DRtg_opponent'] - df['national_avg_DRtg'])
    )
    df['adjD'] = df['national_avg_DRtg'] + alpha * (
        (df['DRtg'] - df['national_avg_DRtg']) + (df['ORtg_opponent'] - df['national_avg_ORtg'])
    )
    df['adjEM'] = df['adjO'] - df['adjD']

    
    return df


def calculate_season_averages(df):
    """
    Calculate season-long averages for all metrics, including expected efficiency margin.

    Parameters:
        df (pd.DataFrame): DataFrame containing game-level metrics.

    Returns:
        pd.DataFrame: Season averages for each team, including exp_em.
    """
    # Group by team and season, then calculate averages
    season_avg = df.groupby(['team_id', 'season']).agg({
        'tempo': 'mean',
        'ORtg': 'mean',
        'DRtg': 'mean',
        'EM': 'mean',
        'adjT': 'mean',
        'adjO': 'mean',
        'adjD': 'mean',
        'adjEM': 'mean',
        # 'exp_em': 'mean'  # Add exp_em to the aggregation
    }).reset_index()

    return season_avg

def calculate_expected_efficiency_margin_all_games(df):
    """
    Calculate the expected efficiency margin for all games in the DataFrame.

    Parameters:
        df (pd.DataFrame): DataFrame containing game-level metrics.

    Returns:
        pd.DataFrame: Original DataFrame with an added column for expected efficiency margin.
    """
    # National averages (can be dynamic or static)
    national_avg_ORtg = df['ORtg'].mean()
    national_avg_DRtg = df['DRtg'].mean()

    # Merge opponent stats
    df = df.merge(
        df[['game_id', 'team_id', 'adjO', 'adjD']],
        how='left',
        left_on=['game_id', 'opponent_team_id'],
        right_on=['game_id', 'team_id'],
        suffixes=('', '_opponent')
    )

    # Home court advantage adjustment (points per 100 possessions)
    home_adjustment = 2

    # Calculate expected offensive efficiency (Exp OE) for the home team
    df['exp_oe'] = df['adjO'] * (national_avg_DRtg / df['adjD_opponent']) + home_adjustment

    # Calculate expected defensive efficiency (Exp DE) for the home team
    df['exp_de'] = df['adjD'] * (df['adjO_opponent'] / national_avg_ORtg) - home_adjustment

    # Calculate expected efficiency margin (Exp EM) for the home team
    df['exp_em'] = df['exp_oe'] - df['exp_de']

    # Clean up intermediate columns
    df.drop(columns=['adjO_opponent', 'adjD_opponent', 'exp_oe', 'exp_de'], inplace=True)

    return df

# Example usage in a Jupyter Notebook
# df = calculate_expected_efficiency_margin_all_games(df)


def calculate_lagged_rolling_mean(df):
    """
    Calculate lagged rolling mean of adjO, adjD, adjEM, ORtg, DRtg, and EM for each game.
    """
    # Sort by team and game_date
    df = df.sort_values(by=['team_id', 'game_date'])

    # Calculate lagged rolling mean for adjO, adjD, adjEM, ORtg, DRtg, and EM
    for col in ['adjO', 'adjD', 'adjEM', 'ORtg', 'DRtg', 'EM']:
        df[f'lagged_{col}'] = df.groupby('team_id')[col].apply(
            lambda x: x.expanding().mean().shift(1)
        ).reset_index(level=0, drop=True)


    
    return df

def merge_opponent_lagged_metrics(df):
    """
    Merge the opponent's lagged adjO, adjD, adjEM, ORtg, DRtg, and EM into the DataFrame.
    """
    # Create a DataFrame with the opponent's lagged metrics
    opponent_lagged = df[['team_id', 'game_date', 'lagged_adjO', 'lagged_adjD', 'lagged_adjEM', 
                          'lagged_ORtg', 'lagged_DRtg', 'lagged_EM']].rename(
        columns={
            'team_id': 'opponent_team_id',
            'lagged_adjO': 'opponent_lagged_adjO',
            'lagged_adjD': 'opponent_lagged_adjD',
            'lagged_adjEM': 'opponent_lagged_adjEM',
            'lagged_ORtg': 'opponent_lagged_ORtg',
            'lagged_DRtg': 'opponent_lagged_DRtg',
            'lagged_EM': 'opponent_lagged_EM'
        }
    )

    # Merge the opponent's lagged metrics into the original DataFrame
    df = df.merge(
        opponent_lagged,
        how='left',
        left_on=['opponent_team_id', 'game_date'],
        right_on=['opponent_team_id', 'game_date']
    )
    df['lagged_EM_diff'] = df['lagged_EM'] - df['opponent_lagged_EM']
    df['lagged_adjEM_diff'] = df['lagged_adjEM'] - df['opponent_lagged_adjEM']
    df['team_score_diff'] = df['team_score'] - df['opponent_team_score']
    return df


def main(df):
    """
    Main function to calculate all metrics and aggregate into season averages.
    """
    # Calculate possessions
    df = calculate_possessions(df)

    # Calculate tempo
    df = calculate_tempo(df)

    # Calculate efficiency (plain ORtg, DRtg, EM)
    df = calculate_efficiency(df)

    # Calculate adjusted metrics
    df = calculate_adjusted_metrics(df)

    df = calculate_lagged_rolling_mean(df)

    df = merge_opponent_lagged_metrics(df)
    # df = calculate_expected_efficiency_margin_all_games(df)

    # Calculate season averages
    season_avg = calculate_season_averages(df)

    return df, season_avg


In [ ]:
##ALPHA 0.25
# from sklearn.metrics import mean_squared_error
# for i in np.arange(0.01, 0.5, 0.01):    
#     kp, season_avg = main(df, i)
#     kp = kp.dropna(subset=['lagged_EM_diff', 'lagged_adjEM_diff', 'team_score_diff'])
#     kp = kp.loc[(kp['team_home_away'] == 'home')]
#     # Calculate RMSE
#     rmse_em = np.sqrt(mean_squared_error(kp['team_score_diff'], kp['lagged_EM_diff']))
#     rmse_adjem = np.sqrt(mean_squared_error(kp['team_score_diff'], kp['lagged_adjEM_diff']))
#     print(f'Alpha: {i}')
#     print(f'RMSE EM: {rmse_em}')
#     print(f'RMSE AdjEM: {rmse_adjem}')





Alpha: 0.01
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.823180230120292
RMSE Adj Laged AdjEM: 13.894338616463873
Alpha: 0.02
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.782652436601532
RMSE Adj Laged AdjEM: 13.871405921925565
Alpha: 0.03
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.743797712662701
RMSE Adj Laged AdjEM: 13.849273818637055
Alpha: 0.04
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.706630286459598
RMSE Adj Laged AdjEM: 13.82794615072035
Alpha: 0.05
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.671163919632269
RMSE Adj Laged AdjEM: 13.807426645893852
Alpha: 0.060000000000000005
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.637411883854044
RMSE Adj Laged AdjEM: 13.787718912392252
Alpha: 0.06999999999999999
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.605386937789621
RMSE Adj Laged AdjEM: 13.768826435953802
Alpha: 0.08
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.575101304547752
RMSE Adj Laged AdjEM: 13.750752576880412
Alpha: 0.09
RMSE EM: 13.81235349559877
RMSE AdjEM: 13.546566649714475
RMSE Adj Lag

In [127]:
kp, season_avg = main(df)

In [128]:
kp

,game_id,season,season_type,game_date,game_date_time,team_id,team_uid,team_slug,team_location,team_name,team_abbreviation,team_display_name,team_short_display_name,team_color,team_alternate_color,team_logo,team_home_away,team_score,team_winner,assists,blocks,defensive_rebounds,field_goal_pct,field_goals_made,field_goals_attempted,flagrant_fouls,fouls,free_throw_pct,free_throws_made,free_throws_attempted,largest_lead,offensive_rebounds,steals,team_turnovers,technical_fouls,three_point_field_goal_pct,three_point_field_goals_made,three_point_field_goals_attempted,total_rebounds,total_technical_fouls,total_turnovers,turnovers,opponent_team_id,opponent_team_uid,opponent_team_slug,opponent_team_location,opponent_team_name,opponent_team_abbreviation,opponent_team_display_name,opponent_team_short_display_name,opponent_team_color,opponent_team_alternate_color,opponent_team_logo,opponent_team_score,fast_break_points,points_in_paint,turnover_points,season_game_n,minutes,days_since_last_game,airport,distance_travelled,days_between_games,distance_per_day,avg_distance_per_day_last_5,possessions,tempo,ORtg,DRtg,EM,national_avg_tempo,national_avg_ORtg,national_avg_DRtg,team_id_opponent,tempo_opponent,ORtg_opponent,DRtg_opponent,adjT,adjO,adjD,adjEM,lagged_adjO,lagged_adjD,lagged_adjEM,lagged_ORtg,lagged_DRtg,lagged_EM,opponent_lagged_adjO,opponent_lagged_adjD,opponent_lagged_adjEM,opponent_lagged_ORtg,opponent_lagged_DRtg,opponent_lagged_EM,lagged_EM_diff,lagged_adjEM_diff,team_score_diff
0,400927396,2017,2,2017-05-14,2017-05-14 18:00:00,3,s:40~l:59~t:3,dallas-wings,Dallas,Wings,DAL,Dallas Wings,Wings,11213e,c5d646,https://a.espncdn.com/i/teamlogos/wnba/500/dal...,away,68,True,9,1,29,41.7,25,60,0,20,85.0,17,20,12.0,12,7,1,0,10.0,1,10,41,0,18,17,11,s:40~l:59~t:11,phoenix-mercury,Phoenix,Mercury,PHX,Phoenix Mercury,Mercury,3a267d,e56020,https://a.espncdn.com/i/teamlogos/wnba/500/phx...,58,NaN,NaN,NaN,1,200.0,NaN,PHX,0,0,0.000000,0.000000,74.500,74.500000,91.275168,77.852349,13.422819,74.500000,91.275168,77.852349,11,75.125000,77.204659,90.515807,74.578125,94.441032,74.334722,20.106311,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
1,400927405,2017,2,2017-05-20,2017-05-20 20:00:00,3,s:40~l:59~t:3,dallas-wings,Dallas,Wings,DAL,Dallas Wings,Wings,11213e,c5d646,https://a.espncdn.com/i/teamlogos/wnba/500/dal...,home,87,False,16,3,17,50.0,31,62,0,25,81.5,22,27,3.0,8,11,1,1,23.1,3,13,25,1,17,16,8,s:40~l:59~t:8,minnesota-lynx,Minnesota,Lynx,MIN,Minnesota Lynx,Lynx,005084,47a757,https://a.espncdn.com/i/teamlogos/wnba/500/min...,89,NaN,NaN,NaN,2,200.0,6.0,MSP,1260,6,210.000000,105.000000,82.825,82.825000,105.040749,107.455478,-2.414730,78.662500,98.157958,92.653914,8,82.150000,108.338405,105.903834,79.618750,103.191136,98.899417,4.291719,94.441032,74.334722,20.106311,91.275168,77.852349,13.422819,97.323749,90.082792,7.240956,93.712831,77.365096,16.347734,-2.924916,12.865354,-2
2,400927412,2017,2,2017-05-25,2017-05-25 12:00:00,3,s:40~l:59~t:3,dallas-wings,Dallas,Wings,DAL,Dallas Wings,Wings,11213e,c5d646,https://a.espncdn.com/i/teamlogos/wnba/500/dal...,away,94,True,22,5,23,44.4,32,72,0,18,86.2,25,29,12.0,12,10,1,0,26.3,5,19,35,0,16,15,17,s:40~l:59~t:17,san-antonio-stars,Las Vegas,Aces,LV,San Antonio Stars,Aces,000000,c7cfd4,NaN,82,NaN,NaN,NaN,3,200.0,5.0,LAS,1320,5,264.000000,158.000000,88.775,88.775000,105.885666,92.368347,13.517319,82.033333,100.733861,92.558725,17,83.925000,97.706285,112.004766,83.112500,106.883322,91.754236,15.129086,98.816084,86.617069,12.199015,98.157958,92.653914,5.504044,95.439041,100.180161,-4.741120,88.000360,100.836845,-12.836485,18.340530,16.940135,12
3,400927417,2017,2,2017-05-27,2017-05-27 22:00:00,3,s:40~l:59~t:3,dallas-wings,Dallas,Wings,DAL,Dallas Wings,Wings,11213e,c5d646,https://a.espncdn.com/i/teamlogos/wnba/500/dal...,away,65,False,10,6,26,32.3,21,65,0,25,70.4,19,27,5.0,8,5,2,1,19.0,4,21,34,1,17,15,11,s:40~l:59~t:11,phoenix-mercury,Phoenix,Mercury,PHX,Phoenix Mercury,Mercury,3a267d,e56020,https://a.espncd

RMSE EM: 13.81235349559877
RMSE AdjEM: 13.37468573126762


In [118]:
team_names = df[['team_id', 'team_name']].drop_duplicates()
season_avg['adjEM'] = season_avg['adjO'] - season_avg['adjD']
season_avg = pd.merge(season_avg, team_names, on='team_id', how='left')
season_avg = season_avg.loc[(season_avg['team_id'] < 90)]
season_avg = season_avg.sort_values(by='adjEM', ascending=False)
season_avg

,team_id,season,tempo,ORtg,DRtg,EM,adjT,adjO,adjD,adjEM,team_name
58,16,2019,79.887582,111.514480,96.554115,14.960365,79.850204,122.745395,92.712757,30.032638,Mystics
51,14,2020,82.369318,106.440583,92.266877,14.173706,82.304545,113.148719,84.782061,28.366658,Storm
70,17,2023,83.333537,110.563584,96.505469,14.058116,83.446037,120.759524,92.766771,27.992754,Aces
24,8,2017,80.304412,106.084668,92.456890,13.627778,80.294485,112.633767,85.196474,27.437292,Lynx
76,18,2021,77.709259,101.126174,89.586543,11.539631,77.155135,103.889536,80.566880,23.322656,Sun
...,...,...,...,...,...,...,...,...,...,...,...
9,5,2018,79.702337,94.323141,105.967111,-11.643970,79.872941,87.113069,110.294257,-23.181188,Fever
8,5,2017,78.596078,94.988745,106.605431,-11.616686,78.431699,90.326818,113.808784,-23.481966,Fever
12,5,2021,80.450000,94.007094,106.043735,-12.036642,80.509375,87.945178,112.087829,-24.142651,Fever
13,5,2022,83.355324,92.969363,106.531590,-13.562227,83.448881,85.850530,112.912707,-27.062177,Fever


In [119]:
season_avg[season_avg['season'] == 2024]

,team_id,season,tempo,ORtg,DRtg,EM,adjT,adjO,adjD,adjEM,team_name
39,9,2024,81.070122,105.601516,94.920049,10.681468,80.838110,111.827693,90.344724,21.482969,Liberty
79,18,2024,79.048194,100.834538,92.647662,8.186876,78.676701,102.626809,86.089852,16.536957,Sun
31,8,2024,79.860163,102.239770,94.404278,7.835492,80.441335,103.043219,87.460702,15.582517,Lynx
71,17,2024,81.817014,105.396023,98.662601,6.733421,82.207500,109.814406,96.445757,13.368648,Aces
55,14,2024,83.042375,99.805107,94.341159,5.463948,82.449625,101.146167,90.127844,11.018323,Storm
15,5,2024,82.334722,102.846658,106.223970,-3.377312,82.827465,104.562951,111.214467,-6.651516,Fever
95,20,2024,80.064167,95.298790,98.718680,-3.419890,80.066840,90.588678,97.475465,-6.886787,Dream
63,16,2024,81.694931,96.610265,100.173478,-3.563213,81.819444,93.054670,100.213363,-7.158693,Mystics
47,11,2024,81.076750,99.973495,104.015356,-4.041861,81.203625,99.620916,107.810317,-8.189400,Mercury
87,19,2024,80.981250,95.575796,102.256196,-6.680399,81.267500,90.502468,103.870346,-13.367878,Sky


In [120]:
kp

,game_id,season,season_type,game_date,game_date_time,team_id,team_uid,team_slug,team_location,team_name,team_abbreviation,team_display_name,team_short_display_name,team_color,team_alternate_color,team_logo,team_home_away,team_score,team_winner,assists,blocks,defensive_rebounds,field_goal_pct,field_goals_made,field_goals_attempted,flagrant_fouls,fouls,free_throw_pct,free_throws_made,free_throws_attempted,largest_lead,offensive_rebounds,steals,team_turnovers,technical_fouls,three_point_field_goal_pct,three_point_field_goals_made,three_point_field_goals_attempted,total_rebounds,total_technical_fouls,total_turnovers,turnovers,opponent_team_id,opponent_team_uid,opponent_team_slug,opponent_team_location,opponent_team_name,opponent_team_abbreviation,opponent_team_display_name,opponent_team_short_display_name,opponent_team_color,opponent_team_alternate_color,opponent_team_logo,opponent_team_score,fast_break_points,points_in_paint,turnover_points,minutes,possessions,tempo,ORtg,DRtg,EM,national_avg_tempo,national_avg_ORtg,national_avg_DRtg,team_id_opponent,tempo_opponent,ORtg_opponent,DRtg_opponent,adjT,adjO,adjD,adjEM
0,400927594,2017,2,2017-09-03,2017-09-03 18:00:00,14,s:40~l:59~t:14,seattle-storm,Seattle,Storm,SEA,Seattle Storm,Storm,05452a,f7c43c,https://a.espncdn.com/i/teamlogos/wnba/500/sea...,away,85,True,22,5,31,39.4,28,71,0,14,88.9,24,27,7.0,9,9,3,2,23.8,5,21,40,2,12,9,19,s:40~l:59~t:19,chicago-sky,Chicago,Sky,CHI,Chicago Sky,Sky,4f91cd,f1f2f3,https://a.espncdn.com/i/teamlogos/wnba/500/chi...,80,NaN,NaN,NaN,200.0,83.825,83.825,101.401730,95.436922,5.964808,83.825000,101.401730,95.436922,19,87.175,91.769429,97.505019,85.5000,103.469826,85.804622,17.665205
1,400927594,2017,2,2017-09-03,2017-09-03 18:00:00,19,s:40~l:59~t:19,chicago-sky,Chicago,Sky,CHI,Chicago Sky,Sky,4f91cd,f1f2f3,https://a.espncdn.com/i/teamlogos/wnba/500/chi...,home,80,False,22,3,32,44.3,31,70,0,21,84.6,11,13,10.0,5,4,0,1,35.0,7,20,37,1,16,16,14,s:40~l:59~t:14,seattle-storm,Seattle,Storm,SEA,Seattle Storm,Storm,05452a,f7c43c,https://a.espncdn.com/i/teamlogos/wnba/500/sea...,85,NaN,NaN,NaN,200.0,87.175,87.175,91.769429,97.505019,-5.735589,85.500000,96.585580,96.470970,14,83.825,101.401730,95.436922,85.5000,90.735381,102.321169,-11.585788
2,400927592,2017,2,2017-09-03,2017-09-03 16:00:00,18,s:40~l:59~t:18,connecticut-sun,Connecticut,Sun,CONN,Connecticut Sun,Sun,de6b36,002d62,https://a.espncdn.com/i/teamlogos/wnba/500/con...,away,70,False,13,1,25,38.1,24,63,0,12,88.2,15,17,0.0,7,6,1,1,46.7,7,15,32,1,18,17,6,s:40~l:59~t:6,los-angeles-sparks,Los Angeles,Sparks,LA,Los Angeles Sparks,Sparks,532481,49ac9c,https://a.espncdn.com/i/teamlogos/wnba/500/la.png,81,NaN,NaN,NaN,200.0,81.075,81.075,86.339809,99.907493,-13.567684,84.025000,93.170323,97.616478,6,82.850,97.767049,84.490042,81.9625,73.213373,104.504219,-31.290846
3,400927592,2017,2,2017-09-03,2017-09-03 16:00:00,6,s:40~l:59~t:6,los-angeles-sparks,Los Angeles,Sparks,LA,Los Angeles Sparks,Sparks,532481,49ac9c,https://a.espncdn.com/i/teamlogos/wnba/500/la.png,home,81,True,21,4,29,51.4,37,72,0,14,100.0,6,6,19.0,5,11,1,0,16.7,1,6,34,0,14,13,18,s:40~l:59~t:18,connecticut-sun,Connecticut,Sun,CONN,Connecticut Sun,Sun,de6b36,002d62,https://a.espncdn.com/i/teamlogos/wnba/500/con...,70,NaN,NaN,NaN,200.0,82.850,82.850,97.767049,84.490042,13.277007,83.731250,94.319504,94.334869,18,81.075,86.339809,99.907493,81.9625,103.339673,76.510347,26.829326
4,400927593,2017,2,2017-09-03,2017-09-03 16:00:00,20,s:40~l:59~t:20,atlanta-dream,Atlanta,Dream,ATL,Atlanta Dream,Dream,d02030,383a37,https://a.espncdn.com/i/teamlogos/wnba/500/atl...,away,70,False,8,1,21,36.9,24,65,0,24,65.2,15,23,3.0,7,3,1,2,38.9,7,18,28,2,11,10,11,s:40~l:59~t:11,phoenix-mercury,Phoenix,Mercury,PHX,Phoenix Mercury,Mercury,3a267d,e56020,https://a.espncdn.com/i/teamlogos/wnba/500/phx...,84,NaN,NaN,NaN,200.0,78.925,78.925,88.691796,106.430155,-17.738359,82.770000,93.193963,96.753926,11,80.350,104.542626,87.118855,79.6375,79.056725,117.778819,-38.722094
...,...,...,...,...,